In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [39]:
survey_data = pd.read_csv('../survey/survey_data.csv')

n = 5

survey_responses = pd.read_csv('FINAL_SMALL.csv')
survey_responses_2 = pd.read_csv('FINAL_SMALL_2.csv')
survey_responses_3 = pd.read_csv('FINAL_SMALL_3.csv')
survey_responses_4 = pd.read_csv('FINAL_SMALL_4.csv')
survey_responses_5 = pd.read_csv('FINAL_SMALL_5.csv')
survey_responses_6 = pd.read_csv('FINAL_SMALL_6.csv')

all_responses = [survey_responses, survey_responses_2, survey_responses_3, survey_responses_4, survey_responses_5, survey_responses_6]


In [42]:
method_names = survey_data.iloc[0][['Method_1', 'Method_2', 'Method_3']]

In [ ]:
survey_data

In [44]:
def get_results(df, success_only=False):
    questions = [x for x in df.columns if '_mc_' in x]
    qids = set([x.split('_')[2] for x in questions])
    
    rows = {qid: {k: [] for k in method_names} for qid in qids}
    for question in questions:
        split = question.split('_')
        qid = split[2]
        method = split[3]
        
        if success_only:
            if not survey_data[survey_data['Question_ID'] == int(qid)].iloc[0][f'{method}_Success']:
                continue

        ratings = df[question].dropna().iloc[2:].to_list()
        for rating in ratings:
            rows[qid][method].append(int(rating))
            
        rows[qid][method] = rows[qid][method][:5]
            
    all_ratings = {k: [] for k in method_names}

    for row in rows:
        for method in rows[row]:
            if len(rows[row][method]) > 0:
                all_ratings[method].append(np.mean(rows[row][method]))
    
    return all_ratings, rows

In [45]:
all_ratings, rows = get_results(survey_responses)

all_ratings_2, rows_2 = get_results(survey_responses_2)

all_ratings_3, rows_3 = get_results(survey_responses_3)

all_ratings_4, rows_4 = get_results(survey_responses_4)

all_ratings_5, rows_5 = get_results(survey_responses_5)

all_ratings_6, rows_6 = get_results(survey_responses_6)

In [48]:
all_all_ratings = [all_ratings, all_ratings_2, all_ratings_3, all_ratings_4, all_ratings_5, all_ratings_6]

In [28]:
success_only = False
all_rows = {}
for responses in all_responses:
    all_ratings, rows = get_results(responses, success_only)
    all_rows |= rows

averaged = {name: [] for name in method_names}
means = {str(i): {name: {} for name in method_names} for i in range(n)}
for i in range(n):
    i = str(i)
    for method in method_names:
        if len(all_rows[i][method]) == 0:
            continue
        
        means[i][method] = {}
        
        means[i][method]['mean'] = np.mean(all_rows[i][method])
        means[i][method]['std'] = np.std(all_rows[i][method])
        
        averaged[method].append(means[i][method]['mean'])
        
all_averages = {}
        
for method in method_names:
    average = np.mean(averaged[method])
    std = np.std(averaged[method])
    
    all_averages[method] = averaged[method]
    
    averaged[method] = {'average': average, 'std': std}

In [ ]:
averaged

**ALL**

| Generator |   Fluency       |
|-----------|-----------------|
| PPLM | 2.86 (0.67)     |
| Polyjuice      | 3.40 (0.94)     |
| RELITC    | **3.43** (0.79) |

**SUCCESSFUL CE ONLY**

| Generator |   Fluency       |
|-----------|-----------------|
| PPLM | 2.96 (0.65)     | 
| Polyjuice      | 3.19 (1.03)     | 
| RELITC    | **3.35** (0.83) |

In [ ]:
all_means = [np.array([np.mean(ratings[x]) for x in ratings]) for ratings in all_all_ratings]
all_stds = [np.array([np.std(ratings[x]) for x in ratings]) for ratings in all_all_ratings]

# means_1 = np.array([np.mean(all_ratings[x]) for x in all_ratings])
# stds_1 = np.array([np.std(all_ratings[x]) for x in all_ratings])

# means_2 = np.array([np.mean(all_ratings_2[x]) for x in all_ratings_2])
# stds_2 = np.array([np.std(all_ratings_2[x]) for x in all_ratings_2])

# means_3 = np.array([np.mean(all_ratings_3[x]) for x in all_ratings_3])
# stds_3 = np.array([np.std(all_ratings_3[x]) for x in all_ratings_3])

# means_4 = np.array([np.mean(all_ratings_4[x]) for x in all_ratings_4])
# stds_4 = np.array([np.std(all_ratings_4[x]) for x in all_ratings_4])

means = np.mean(all_means, axis=0)
stds = np.mean(all_stds, axis=0)

xs = np.arange(len(method_names))

plt.grid(axis='y', linestyle="--", alpha=0.5, zorder=1)

offset = 1 / (len(all_means) + 2)

for i, (mean, std) in enumerate(zip(all_means, all_stds)):
    plt.bar(xs+(offset*i), mean, yerr=(std), width=offset, label=f'Batch {i+1}', capsize=5)

# plt.bar(xs, means_1, yerr=(stds_1), width=offset, label='1st Batch', capsize=5)
# plt.bar(xs+offset, means_2, yerr=(stds_2), width=offset, label='2nd Batch', capsize=5)
# plt.bar(xs+2*offset, means_3, yerr=(stds_3), width=offset, label='3rd Batch', capsize=5)
# plt.bar(xs+3*offset, means_4, yerr=(stds_4), width=offset, label='4th Batch', capsize=5)
plt.bar(xs+offset*(len(means)+3), means, yerr=(stds), width=offset, label='Mean', capsize=5)

plt.xticks(xs + offset * (len(means) / 2 + 1), method_names)
plt.title('Four batches of ratings')
plt.legend()
plt.savefig('fourth_batch.png')
plt.show()

In [ ]:
np.mean(all_means, axis=0)

In [ ]:
all_means[5]

In [ ]:
for method in all_ratings:
    print(method, np.mean(all_ratings[method]))

In [ ]:
means

In [17]:
all_responses = [survey_responses, survey_responses_2, survey_responses_3, survey_responses_4, survey_responses_5, survey_responses_6]

In [18]:
all_questions = [[x for x in responses.columns if '_mc_' in x] for responses in all_responses]

In [19]:
questions_1 = [x for x in survey_responses.columns if '_mc_' in x]
questions_2 = [x for x in survey_responses_2.columns if '_mc_' in x]

In [20]:
all_ratings_per_person = [[row.dropna().astype(int).to_list() for i, row in responses.iloc[2:][questions].iterrows() if len(row.dropna()) > 0] for responses, questions in zip(all_responses, all_questions)]

In [21]:
ratings_per_person_1 = [row.dropna().astype(int).to_list() for i, row in survey_responses.iloc[2:][questions_1].iterrows() if len(row.dropna()) > 0]

In [22]:
ratings_per_person_2 = [row.dropna().astype(int).to_list() for i, row in survey_responses_2.iloc[2:][questions_2].iterrows() if len(row.dropna()) > 0]

In [23]:
means_pp = [[np.mean(x) for x in ratings_per_person] for ratings_per_person in all_ratings_per_person]
stds_pp = [[np.std(x) for x in ratings_per_person] for ratings_per_person in all_ratings_per_person]

In [41]:
means_pp_1 = [np.mean(x) for x in ratings_per_person_1]
means_pp_2 = [np.mean(x) for x in ratings_per_person_2]
stds_pp_1 = [np.std(x) for x in ratings_per_person_1]
stds_pp_2 = [np.std(x) for x in ratings_per_person_2]

In [ ]:
plt.grid(axis='y', linestyle="--", alpha=0.5, zorder=1)

for i in range(len(means_pp)):
    num_responses = len(means_pp[i])
    plt.scatter([i+1]*num_responses, means_pp[i])
    if i == 0:
        plt.scatter([i+1]*num_responses, stds_pp[i], color='black', label='SD')
        plt.scatter([i+1], np.mean(means_pp[i]), color='blue', label='Mean')
        plt.scatter([i+1], np.mean(stds_pp[i]), color='red', label='Mean SD')
    else:
        plt.scatter([i+1]*num_responses, stds_pp[i], color='black')
        plt.scatter([i+1], np.mean(means_pp[i]), color='blue')
        plt.scatter([i+1], np.mean(stds_pp[i]), color='red')
        

# plt.scatter([1]*5, means_pp_1)
# plt.scatter([2]*5, means_pp_2)
# plt.scatter([1]*5, stds_pp_1, label='Std', color='black')
# plt.scatter([2]*5, stds_pp_2, color='black')

ticks = [f'Batch {i}' for i in range(1, len(means_pp) + 1)]
xs = np.arange(len(ticks))
plt.xticks(xs+1, ticks)

plt.legend()
plt.ylim(-0.2, 5.2)
plt.xlim(0, 7)
plt.title('Ratings per respondent (mean, std)')
plt.savefig('respondents_ratings.png')

In [ ]:
fig, ax = plt.subplots()
bp1 = ax.violinplot(means_pp, widths=0.5, showmeans=True)
bp2 = ax.violinplot(stds_pp, widths=0.5, showmeans=True)

for pc in bp1['bodies']:
    pc.set_facecolor('aliceblue')
    pc.set_edgecolor('tab:blue')
    pc.set_alpha(1)
    
for partname in ('cbars','cmins','cmaxes','cmeans'):
    vp = bp1[partname]
    vp.set_edgecolor('tab:blue')
    vp.set_linewidth(1)
    
for pc in bp2['bodies']:
    pc.set_facecolor('lightgrey')
    pc.set_edgecolor('black')
    pc.set_alpha(1)
    
for partname in ('cbars','cmins','cmaxes','cmeans'):
    vp = bp2[partname]
    vp.set_edgecolor('black')
    vp.set_linewidth(1)


ax.legend([bp1["bodies"][0], bp2["bodies"][0]], ['Ratings', 'SD'], loc='upper right')

ticks = [f'Batch {i}' for i in range(1, len(means_pp) + 1)]
xs = np.arange(len(ticks))
plt.xticks(xs + 1, ticks)

plt.grid(axis='y', linestyle="--", alpha=0.5, zorder=1)

plt.ylim(-0.2, 5.2)
plt.xlim(0.5, 6.5)

plt.title('Ratings per respondent')

plt.savefig('respondents_ratings_violin.png')

In [ ]:
fig, ax = plt.subplots()
bp1 = ax.boxplot(means_pp, widths=0.35, patch_artist=True, boxprops=dict(facecolor="C0"))
bp2 = ax.boxplot(stds_pp, widths=0.35, patch_artist=True, boxprops=dict(facecolor="C2"))

ax.legend([bp1["boxes"][0], bp2["boxes"][0]], ['Ratings', 'SD'], loc='upper right')

ticks = [f'Batch {i}' for i in range(1, len(means_pp) + 1)]
xs = np.arange(len(ticks))
plt.xticks(xs+1, ticks)
plt.grid(axis='y', linestyle="--", alpha=0.5, zorder=1)


plt.ylim(-0.2, 5.2)
plt.xlim(0, 7)
plt.title('Ratings per respondent')
plt.savefig('respondents_ratings_box.png')

In [ ]:
np.mean(stds_pp_1), np.mean(stds_pp_2)

In [ ]:
np.mean(means_pp_1), np.mean(means_pp_2)

In [ ]:
plt.scatter(all_averages['RELITC'], all_averages['Polyjuice'], c=all_averages['PPLM'], alpha=0.6)
plt.colorbar()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(all_averages['RELITC'], all_averages['Polyjuice'], all_averages['PPLM'])

In [56]:
id_to_fomc = survey_data[['Question_ID', 'Fact_ID']].drop_duplicates()

indexes = all_rows.keys()
fomc_indexes = [int(id_to_fomc[id_to_fomc['Question_ID'] == int(ix)]['Fact_ID'].iloc[0]) for ix in indexes]

polyjuice = [np.mean(all_rows[ix]['Polyjuice']) for ix in indexes]
pplm = [np.mean(all_rows[ix]['PPLM']) for ix in indexes]
relitc = [np.mean(all_rows[ix]['RELITC']) for ix in indexes]

In [57]:
df = pd.DataFrame({
    'q_id': indexes,
    'id': fomc_indexes,
    'polyjuice': polyjuice,
    'pplm': pplm,
    'relitc': relitc,
})

In [60]:
df.to_csv('non-expert_mean_results.csv')

In [ ]:
for means in means_pp: # intra-rater - means of means of person's ratings
    print(np.mean(means), np.std(means))

In [ ]:
for ratings in all_all_ratings: # inter-rater (means of means of all the ratings in batch)
    agg = []
    for name in ratings:
        agg += ratings[name]
    print(np.mean(agg), np.std(agg))

In [ ]:
all_all_ratings[3]

In [ ]:
means_pp